In [18]:
!python -m pip install -U gensim --user

In [19]:
import sklearn
from sklearnex import patch_sklearn 
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [20]:
import pandas as pd
import numpy as np
import pickle
import gensim
from gensim import corpora, models

from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

from utils import *

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
train = pd.read_csv('../data/FullLitCovid/train.tsv',sep='\t',header=None,dtype={0:str,1:str})
val = pd.read_csv('../data/FullLitCovid/val.tsv',sep='\t',header=None,dtype={0:str,1:str})
test = pd.read_csv('../data/FullLitCovid/test.tsv',sep='\t',header=None,dtype={0:str,1:str})
train.head(5)

,0,1
0,01100000,structural conservation among variants sars-co...
1,00001000,effective management idiopathic intracranial h...
2,00001000,`` 's whole different atmosphere '' qualitativ...
3,01100000,modification spike protein vaccines enveloped ...
4,00001010,analysis prediction covid-19 outbreak pakistan...


In [22]:

train[0].value_counts()

00001000    8040
01100000    3082
11000000    2835
01000000    2653
10000000    2609
00000100    1996
00100000     549
11100000     469
00011000     440
10001000     419
01001000     267
00001010     224
10100000     182
00000010     134
00110000     112
00010000     109
11001000      84
10010000      78
00000001      77
10011000      69
00011010      46
01110000      29
00101000      22
00111000      22
11010000      21
11110000      18
11101000      17
01101000      15
01010000      14
01011000      12
10110000      12
00010010      11
01001010       8
11111000       8
01000010       7
11011000       6
10111000       5
10101000       4
01111000       4
10000010       3
00110010       2
10001010       1
01011010       1
00100010       1
01010010       1
Name: 0, dtype: int64

In [23]:
test[0].value_counts()

00001000    2367
01100000     889
11000000     786
01000000     726
10000000     714
00000100     557
00100000     165
11100000     143
00011000     122
10001000     116
01001000      68
00001010      65
10100000      55
00000010      38
00110000      36
11001000      34
00010000      27
10010000      27
00000001      24
00011010      13
10011000      12
01110000      10
11010000      10
00101000       8
11110000       7
00010010       7
00111000       6
10110000       6
01001010       5
11011000       4
11101000       4
01101000       3
01010000       3
10000010       2
11111000       1
01011000       1
01000010       1
01111000       1
Name: 0, dtype: int64

In [24]:
def vectorize_labels(df):
    y = []

    for label in df[0]:
        label_vec = []

        for cat in label:
            label_vec.append(int(cat))


        y.append(np.array(label_vec))
        
    return y

In [25]:
def train_dt(train, val, test):

    tfidf_vectorizer = TfidfVectorizer()

    train_X = tfidf_vectorizer.fit_transform(train[1])
    train_y = vectorize_labels(train)
    

    val_X = tfidf_vectorizer.transform(val[1])
    val_y = vectorize_labels(val)

    test_X = tfidf_vectorizer.transform(test[1])
    test_y = vectorize_labels(test)

    # Create Decision Tree classifer object
    clf = DecisionTreeClassifier()

    # Train Decision Tree Classifer
    clf.fit(train_X, train_y)

    val_pred = clf.predict(val_X)
    test_pred = clf.predict(test_X)
    
    return clf, val_y, test_y, val_pred, test_pred

In [26]:
clf, val_y, test_y, val_pred, test_pred = train_dt(train, val, test)

In [27]:
def eval_model_on_file(clsf, train_file, test_file):
    train = pd.read_csv(train_file,sep='\t',header=None,dtype={0:str,1:str})
    test = pd.read_csv(test_file,sep='\t',header=None,dtype={0:str,1:str})
    
    tfidf_vectorizer = TfidfVectorizer()

    train_X = tfidf_vectorizer.fit_transform(train[1])
    
    test_X = tfidf_vectorizer.transform(test[1])
    test_y = vectorize_labels(test)
    
    test_pred = clsf.predict(test_X)
   
    print(metrics.accuracy_score(test_y, test_pred),metrics.f1_score(test_y, test_pred, average='micro'))
    
    return test_pred

In [28]:
eval_pred = eval_model_on_file(clf, '../data/FullLitCovid/train.tsv','../data/FullLitCovid/val.tsv')

0.5580407701019252 0.7248541901156246


In [29]:
eval_pred_test = eval_model_on_file(clf, '../data/FullLitCovid/train.tsv','../data/FullLitCovid/test.tsv')

0.5629335976214074 0.7242794132731563


In [30]:
eval_model_on_file(clf, '../data/FullLitCovid/train.tsv','../data/cord19_test.tsv')

0.18 0.5472312703583062


array([[1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [1,

In [31]:
#with entropy
def train_dt1(train, val, test):

    tfidf_vectorizer = TfidfVectorizer()

    train_X = tfidf_vectorizer.fit_transform(train[1])
    train_y = vectorize_labels(train)
    

    val_X = tfidf_vectorizer.transform(val[1])
    val_y = vectorize_labels(val)

    test_X = tfidf_vectorizer.transform(test[1])
    test_y = vectorize_labels(test)

    # Create Decision Tree classifer object
    clf = DecisionTreeClassifier(criterion="entropy", max_depth=None)

    # Train Decision Tree Classifer
    clf.fit(train_X, train_y)

    val_pred = clf.predict(val_X)
    test_pred = clf.predict(test_X)
    
    return clf, val_y, test_y, val_pred, test_pred

In [32]:
clf1, val_y, test_y, val_pred, test_pred = train_dt1(train, val, test)

In [33]:
eval_pred = eval_model_on_file(clf1, '../data/FullLitCovid/train.tsv','../data/FullLitCovid/val.tsv')

0.5566251415628539 0.7230531201650336


In [34]:
eval_pred_test = eval_model_on_file(clf, '../data/FullLitCovid/train.tsv','../data/FullLitCovid/test.tsv')

0.5629335976214074 0.7242794132731563


In [35]:
eval_model_on_file(clf1, '../data/FullLitCovid/train.tsv','../data/cord19_test.tsv')

0.2 0.5205479452054794


array([[0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0],
       [1,